In [2]:
!pip install catboost
!pip install scikit-learn --upgrade

In [3]:
# If you have installation questions, please reach out

import pandas as pd # data storage
import catboost as cats # graident boosting 

from catboost import CatBoostRegressor, Pool

import datetime
import numpy as np  # math and stuff
import matplotlib.pyplot as plt # plotting utility
import sklearn # ML and stats

print('catboost ver:', cats.__version__)
print('scikit ver:', sklearn.__version__)

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import max_error, mean_squared_error, median_absolute_error

catboost ver: 0.26.1
scikit ver: 0.24.2


In [4]:

df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS0_Merged_dataset_imputed_08_23_2021.csv')

df2 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS1_Merged_dataset_imputed_08_23_2021.csv')

df3 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS2_Merged_dataset_imputed_08_23_2021.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

print(df.columns.values) # printing all column names

df.describe()

['depth_ft' 'USGS_ID' 'CAL' 'GR' 'DT' 'SP' 'DENS' 'PE' 'RESD' 'PHIN'
 'PHID' 'PE_smooth' 'GR_smooth' 'gz_linear_interp' 'gz_pchip_interp'
 'perm_air_md' 'perm_klink_md' 'He_por' 'horz_perm_md' 'effective_por'
 'por_percent' 'stress_swanson_perm_md' 'sample' 'Na' 'Na Err' 'Mg'
 'Mg Err' 'Al' 'Al Err' 'Si' 'Si Err' 'P' 'P Err' 'S' 'S Err' 'K' 'K Err'
 'Ca' 'Ca Err' 'Ti' 'Ti Err' 'Cr' 'Cr Err' 'Mn' 'Mn Err' 'Fe' 'Fe Err'
 'Co' 'Co Err' 'Ni' 'Ni Err' 'Cu' 'Cu Err' 'Zn' 'Zn Err' 'Ba' 'Ba Err']


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,508.000000,510.000000,510.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5397.863358,0.468406,0.325999,0.430844,0.811259,0.613371,0.351642,1.011018,0.172737,0.172737,0.351702,0.326029,4.913576,4.910244,81.002187,73.797216,14.267039,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5106.156677,0.191316,0.237404,0.263514,0.162843,0.214604,0.186912,1.503608,0.226664,0.226664,0.178735,0.229901,1.738128,1.751499,173.917355,161.251984,9.365678,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003752,0.000000,0.000000,0.000000,-0.744314,0.000570,0.000570,0.004477,0.000258,1.204296,1.199972,0.001000,0.000000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.725000,0.360072,0.122318,0.174146,0.743878,0.424460,0.216157,-0.037336,0.004595,0.004595,0.220800,0.131058,3.538724,3.531571,0.040750,0.020250,8.300000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1614.650000,0.554827,0.275825,0.471657,0.867010,0.605516,0.329327,0.795008,0.005532,0.005532,0.337049,0.283192,4.307541,4.256191,0.065500,0.037500,9.300000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9649.575000,0.602566,0.530980,0.662807,0.925012,0.830179,0.470706,1.378387,0.332503,0.332503,0.463317,0.525448,6.615175,6.633493,71.375000,59.675000,26.610000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.0

In [6]:
dataset = df[[
    'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
    'RESD', 'PHIN', 'PHID', 'GR_smooth', 
    'PE_smooth',
    'gz_pchip_interp' # Trying to predict gz_pchip_interp
]]

dataset.replace('NaN',np.nan, regex=True, inplace=True)# Should be good already
# dataset = dataset.dropna() # not needed

dataset.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,PE_smooth,gz_pchip_interp
0,0.630495,0.187403,0.054025,0.582081,0.829176,0.349137,0.982069,0.001692,0.001692,0.183350,0.366294,4.0320
1,0.629618,0.186340,0.054025,0.583138,0.828532,0.346560,0.970108,0.001674,0.001674,0.185719,0.364550,3.9834
2,0.628741,0.185373,0.054025,0.584257,0.827725,0.349993,0.958103,0.001662,0.001662,0.188207,0.362606,3.9836


In [7]:
# Features we will use for prediction
X = dataset[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y = dataset[['gz_pchip_interp']]

Y_array = np.array(Y.values)

In [8]:
seed = 42 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y_array, test_size=test_size)

In [9]:
preds = 0
preds2 = 0

def catboost_gz(X_train, X_test, y_train, y_test, export_name, max_iter=500):

  model = CatBoostRegressor(objective='RMSE',
                            task_type='GPU',
                            iterations=max_iter)
  
  model.fit(X_train, y_train, verbose=1000 )
  preds = model.predict(X_test)

  # Stats
  rmse = mean_squared_error(y_test, preds, squared=False)
  print("Root Mean Squared Error: %f" % (rmse))
  max = max_error(y_test, preds)
  print("Max Error: %f" % (max))
  MAE= median_absolute_error(y_test, preds)
  print("Median Abs Error: %f" % (max))

  # Let's Tune
  grid = {'learning_rate': [0.1, 0.2, 0.4],
        'depth': [6, 8, 10,  12, 14],
        'l2_leaf_reg': [2, 3, 4 , 5, 6 ]}

  model_grid = CatBoostRegressor(objective='RMSE',
                                 task_type='GPU',
                                 iterations=max_iter)

  # Grid Search
  grid_search_result = model_grid.grid_search(grid, 
                                            X=X_train, 
                                            y=y_train, 
                                            cv=5,
                                            verbose=False)
  model2 = CatBoostRegressor(objective='RMSE',
                           depth=grid_search_result['params']['depth'],
                           l2_leaf_reg=grid_search_result['params']['l2_leaf_reg'],
                           learning_rate=grid_search_result['params']['learning_rate'],
                           iterations=max_iter)

  model2.fit(X_train, y_train, verbose=500)
  preds2 = model2.predict(X_test)

  # Stats for the second round
  rmse2 = mean_squared_error(y_test, preds2, squared=False)
  print("Root Mean Squared Error: %f" % (rmse2))
  max2 = max_error(y_test, preds2)
  print("Max Error: %f" % (max2))
  MAE2= median_absolute_error(y_test, preds2)
  print("Median Abs Error: %f" % (MAE2))

  x = datetime.datetime.now()

  d = {'target': [Y.columns.values, Y.columns.values],
     'MSE': [rmse, rmse2],
     'MAE': [MAE, MAE2],
     'MaxError': [max, max2], 
     'iter': [max_iter, max_iter],
     'day': [x.day, x.day], 
     'month':[x.month, x.month], 
     'year':[x.year, x.year],
     'model':['catboost', 'catboost'],
     'version':[cats.__version__, cats.__version__ ]}

  results = pd.DataFrame(data=d)

  filepath = 'drive/My Drive/1_lewis_research/analysis/experiments/catboost/catboost_results/'
  results.to_csv(filepath+export_name)

  return results

# Iteration

In [10]:
iter = 300

# offset 0

In [11]:
catboost_gz(X_train, X_test, y_train, y_test, 'catboost_gz.csv', iter)

Streaming output truncated to the last 5000 lines.
137:	learn: 0.6316243	test: 0.7705156	best: 0.7705156 (137)	total: 20.3s	remaining: 23.8s
138:	learn: 0.6308410	test: 0.7693874	best: 0.7693874 (138)	total: 20.5s	remaining: 23.7s
139:	learn: 0.6295244	test: 0.7692838	best: 0.7692838 (139)	total: 20.7s	remaining: 23.6s
140:	learn: 0.6283281	test: 0.7693242	best: 0.7692838 (139)	total: 20.9s	remaining: 23.5s
141:	learn: 0.6276491	test: 0.7691418	best: 0.7691418 (141)	total: 21.1s	remaining: 23.4s
142:	learn: 0.6272445	test: 0.7691484	best: 0.7691418 (141)	total: 21.3s	remaining: 23.3s
143:	learn: 0.6270652	test: 0.7690169	best: 0.7690169 (143)	total: 21.5s	remaining: 23.2s
144:	learn: 0.6251735	test: 0.7684790	best: 0.7684790 (144)	total: 21.6s	remaining: 23.1s
145:	learn: 0.6251723	test: 0.7684870	best: 0.7684790 (144)	total: 21.7s	remaining: 22.8s
146:	learn: 0.6242143	test: 0.7683278	best: 0.7683278 (146)	total: 21.8s	remaining: 22.7s
147:	learn: 0.6241314	test: 0.7682456	best: 0.768

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[gz_pchip_interp],0.743674,0.287666,4.683138,300,23,8,2021,catboost,0.26.1
1,[gz_pchip_interp],0.656412,0.142229,4.764305,300,23,8,2021,catboost,0.26.1


# offset 1

In [12]:
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

dataset2 = df2[[
    'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
    'RESD', 'PHIN', 'PHID', 'GR_smooth', 
    'PE_smooth',
    'gz_pchip_interp' # Trying to predict gz_pchip_interp
]]

# Features we will use for prediction
X2 = dataset2[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y = dataset2[['gz_pchip_interp']]

Y_array2 = np.array(Y.values)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2.values, Y_array2, test_size=test_size)

In [13]:
catboost_gz(X_train2, X_test2, y_train2, y_test2, 'OS1_catboost_gz.csv', iter)

Streaming output truncated to the last 5000 lines.
137:	learn: 0.5919076	test: 0.7633344	best: 0.7633344 (137)	total: 20.5s	remaining: 24s
138:	learn: 0.5916605	test: 0.7632147	best: 0.7632147 (138)	total: 20.7s	remaining: 23.9s
139:	learn: 0.5912156	test: 0.7630488	best: 0.7630488 (139)	total: 20.9s	remaining: 23.8s
140:	learn: 0.5912152	test: 0.7630410	best: 0.7630410 (140)	total: 20.9s	remaining: 23.5s
141:	learn: 0.5911285	test: 0.7631789	best: 0.7630410 (140)	total: 21.1s	remaining: 23.4s
142:	learn: 0.5909668	test: 0.7633806	best: 0.7630410 (140)	total: 21.3s	remaining: 23.3s
143:	learn: 0.5907332	test: 0.7633491	best: 0.7630410 (140)	total: 21.5s	remaining: 23.2s
144:	learn: 0.5906383	test: 0.7634737	best: 0.7630410 (140)	total: 21.6s	remaining: 23.1s
145:	learn: 0.5906360	test: 0.7634780	best: 0.7630410 (140)	total: 21.7s	remaining: 22.9s
146:	learn: 0.5902816	test: 0.7628489	best: 0.7628489 (146)	total: 21.9s	remaining: 22.7s
147:	learn: 0.5902470	test: 0.7627436	best: 0.76274

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[gz_pchip_interp],0.726201,0.220678,4.716918,300,23,8,2021,catboost,0.26.1
1,[gz_pchip_interp],0.652704,0.109985,4.809584,300,23,8,2021,catboost,0.26.1


# Offset 2

In [14]:
df3 = df3.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

dataset3 = df3[[
    'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
    'RESD', 'PHIN', 'PHID', 'GR_smooth', 
    'PE_smooth',
    'gz_pchip_interp' # Trying to predict gz_pchip_interp
]]

# Features we will use for prediction
X3 = dataset3[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y3 = dataset3[['gz_pchip_interp']]

Y_array3 = np.array(Y3.values)

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3.values, Y_array3, test_size=test_size)

In [15]:
catboost_gz(X_train3, X_test3, y_train3, y_test3, 'OS2_catboost_gz.csv', iter)

Streaming output truncated to the last 5000 lines.
137:	learn: 0.5543456	test: 0.7849172	best: 0.7849172 (137)	total: 22.3s	remaining: 26.2s
138:	learn: 0.5538787	test: 0.7848118	best: 0.7848118 (138)	total: 22.5s	remaining: 26.1s
139:	learn: 0.5520784	test: 0.7843597	best: 0.7843597 (139)	total: 22.7s	remaining: 25.9s
140:	learn: 0.5520621	test: 0.7844290	best: 0.7843597 (139)	total: 22.9s	remaining: 25.8s
141:	learn: 0.5514321	test: 0.7838653	best: 0.7838653 (141)	total: 23.1s	remaining: 25.7s
142:	learn: 0.5513113	test: 0.7838959	best: 0.7838653 (141)	total: 23.2s	remaining: 25.4s
143:	learn: 0.5509587	test: 0.7837381	best: 0.7837381 (143)	total: 23.3s	remaining: 25.3s
144:	learn: 0.5500396	test: 0.7829746	best: 0.7829746 (144)	total: 23.5s	remaining: 25.2s
145:	learn: 0.5495707	test: 0.7828473	best: 0.7828473 (145)	total: 23.7s	remaining: 25s
146:	learn: 0.5484879	test: 0.7819849	best: 0.7819849 (146)	total: 23.9s	remaining: 24.9s
147:	learn: 0.5484007	test: 0.7818726	best: 0.78187

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[gz_pchip_interp],0.679853,0.174280,3.796243,300,24,8,2021,catboost,0.26.1
1,[gz_pchip_interp],0.616315,0.099653,4.141731,300,24,8,2021,catboost,0.26.1
